In [5]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
# Nixtla TimeGPT client
from nixtla import NixtlaClient

In [6]:
load_dotenv()

TIMEGPT_API_KEY = os.getenv('TIMEGPT_API_KEY')
# DATA_CSV = "history_data.csv"

client = NixtlaClient(
    api_key=TIMEGPT_API_KEY
)

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/air_passengers.csv')
timegpt_fcst_df = client.forecast(df=df, h=12, freq='MS', time_col='timestamp', target_col='value')
timegpt_fcst_df.head()

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,timestamp,TimeGPT
0,1961-01-01,437.83792
1,1961-02-01,426.06270
2,1961-03-01,463.11655
3,1961-04-01,478.24450
4,1961-05-01,505.64648


In [12]:
dtypes_dict = df.dtypes.to_dict()
print(dtypes_dict)

{'timestamp': dtype('O'), 'value': dtype('int64')}


In [96]:
model = NixtlaClient(
    api_key=TIMEGPT_API_KEY
)

In [99]:
# Example: read from CSV
df = pd.read_csv("data/synthetic_orders_for_training.csv", parse_dates=["Date"])

# Quick look
print(df.head())


   OrderID       Date  Year  Month DayOfWeek RegionID ClientID  NumberOfItems  \
0     1000 2025-02-01  2025      2  Saturday       R3     C123              7   
1     1001 2025-02-09  2025      2    Sunday       R2     C789              7   
2     1002 2025-05-11  2025      5    Sunday       R2     C123              9   
3     1003 2025-06-02  2025      6    Monday       R1     C333              1   
4     1004 2025-05-09  2025      5    Friday       R3     C789              6   

   TotalQuantity  AvgProductPrice  DistinctCategories  MaterialDiversity  \
0             13            47.64                   2                  2   
1             23            33.53                   5                  7   
2             42            25.92                   2                  3   
3              1            49.59                   1                  4   
4             28            34.07                   1                  1   

     Status  TotalPrice  
0  Returned      567.33  
1   

In [100]:
df_timegpt = df.rename(
    columns={
        "Date": "ds",
        "TotalPrice": "y"
    }
)

# If you only have one overall time series:
df_timegpt["unique_id"] = "series_1"

# Example of extra columns as exogenous variables:
#   DistinctCategories, MaterialDiversity, Status, RegionID, ...
# TimeGPT will automatically treat all extra columns (beyond ds, y, unique_id) as exogenous.
# Make sure the data is sorted by date.
df_timegpt = df_timegpt[["unique_id", "ds", "y",
                         "DistinctCategories", "MaterialDiversity",
                         "Status", "RegionID", "ClientID", 
                         "NumberOfItems", "TotalQuantity", 
                         "AvgProductPrice"]].sort_values("ds")

df_timegpt.head()


,unique_id,ds,y,DistinctCategories,MaterialDiversity,Status,RegionID,ClientID,NumberOfItems,TotalQuantity,AvgProductPrice
370,series_1,2025-01-02,36.54,3,4,Shipped,R4,C123,1,3,10.42
140,series_1,2025-01-03,51.73,1,3,Shipped,R2,C123,1,3,18.50
230,series_1,2025-01-03,130.21,2,4,Shipped,R1,C456,1,3,37.52
437,series_1,2025-01-04,730.07,2,5,Shipped,R3,C456,6,21,37.57
308,series_1,2025-01-04,303.11,5,8,Open,R1,C456,6,8,43.79


In [103]:
df_timegpt = df_timegpt.reset_index(drop=True)

In [105]:
# Example: sum up duplicates
df_timegpt = (
    df_timegpt
    .groupby(["unique_id", "ds"], as_index=False)
    .agg({
        "y": "sum",
        "DistinctCategories": "sum",  # or "mean", or something that makes sense
        "MaterialDiversity": "mean",
        "Status": "first",
        # etc.
    })
)

In [106]:
# Forecast horizon
H = 14  # e.g., predict 14 days ahead

forecasts = model.forecast(
    df_timegpt,  # ✅ Pass the DataFrame as the first argument
    h=14,
    freq='D'
)

INFO:nixtlats.nixtla_client:Validating inputs...
INFO:nixtlats.nixtla_client:Preprocessing dataframes...
INFO:nixtlats.nixtla_client:Attempt 1 failed...
INFO:nixtlats.nixtla_client:Attempt 2 failed...
INFO:nixtlats.nixtla_client:Attempt 3 failed...
INFO:nixtlats.nixtla_client:Attempt 4 failed...
INFO:nixtlats.nixtla_client:Attempt 5 failed...
INFO:nixtlats.nixtla_client:Attempt 6 failed...


LocalProtocolError: Illegal header value b'Bearer '